# Airbnb Amsterdam Analysis

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pyspark.sql.functions as f

# Importing libraries
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pyspark.ml.feature import VectorAssembler,StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import Bucketizer
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import from_unixtime

In [3]:
df = sqlContext.read.format('com.databricks.spark.csv').options(header='true',inferschema='true').option('escape','"').load('/FileStore/tables/listingsfinal__1_-92842.csv')
df_viz=spark.sql("select * from listingsfinal_csv")


#Vizualizing Data for a better understanding

In [5]:
%sql

select count(*) as Number,neighbourhood_cleansed from listingsfinal_csv group by neighbourhood_cleansed order by Number desc limit 12


Number,neighbourhood_cleansed
3391,De Baarsjes - Oud-West
2477,De Pijp - Rivierenbuurt
2186,Centrum-West
1744,Centrum-Oost
1471,Westerpark
1407,Zuid
1323,Oud-Oost
1152,Bos en Lommer
972,Oostelijk Havengebied - Indische Buurt
609,Oud-Noord


In [6]:
%sql
select count(*) as Number,property_type from listingsfinal_csv group by property_type order by Number desc limit 5


Number,property_type
15227,Apartment
1673,House
677,Townhouse
418,Bed and breakfast
378,Loft


In [7]:
%sql
select count(*) as Number,room_type from listingsfinal_csv group by room_type order by Number desc limit 4


Number,room_type
15579,Entire home/apt
4059,Private room
328,Hotel room
58,Shared room


In [8]:
%sql
select count(*) as Number,accommodates from listingsfinal_csv group by accommodates order by Number desc limit 4


Number,accommodates
10911,2
6071,4
1577,3
499,1


In [9]:
%sql
select count(*) as Number,bathrooms from listingsfinal_csv group by bathrooms order by Number desc limit 5


Number,bathrooms
14633,1
3898,1.5
933,2
259,2.5
102,3


In [10]:
%sql
select count(*) as Number,bedrooms from listingsfinal_csv group by bedrooms order by Number desc limit 3


Number,bedrooms
11877,1
4824,2
1651,3


In [11]:
%sql
select count(*) as Number,beds from listingsfinal_csv group by beds order by Number desc limit 5


Number,beds
10907,1
5047,2
1866,3
1258,4
327,5


In [12]:

df1=sqlContext.read.load("/FileStore/tables/listings.csv",format='com.databricks.spark.csv', header='true', inferSchema='true')

df1 = df1.select("host_since", from_unixtime(unix_timestamp("host_since", 'MM/dd/yyy')).alias('DateFormat'))

from pyspark.sql.functions import month
df1 = df1.withColumn('StartMonth',month(df1.DateFormat))
pdf=df1.toPandas()

ax = pdf.plot.hist(bins=12, alpha=0.5)

#### From the above plot we can see that majority of the new hosts are added in the months of june and july. (SUMMER TIME IN AMSTERDAM)

#Deriving new columns

In [15]:
df1=sqlContext.read.format('com.databricks.spark.csv').options(header='true',inferschema='true').option('escape','"').load('/FileStore/tables/listings.csv')
#df1.printSchema()
df1 = df1.withColumn('amenities', regexp_replace('amenities', '"', ''))
df1 = df1.withColumn('amenities', regexp_replace('amenities', 'Cable TV', 'TV'))
df1 = df1.withColumn('amenities', regexp_replace('amenities', 'TV,TV', 'TV,'))
df1 = df1.withColumn('amenities', regexp_replace('amenities', 'TV,,', 'TV,'))

 
df1_amenities=df1.select('amenities')

amenity=df1_amenities.withColumn('word', f.explode(f.split(f.col('amenities'), ',')))\
    .groupBy('word')\
    .count()\
    .sort('count', ascending=False)
type(amenity)
amenity=amenity.limit(5)

 


from matplotlib import pyplot 
%matplotlib inline
#%matplotlib plot
amenity_list=amenity.collect()
#amenity_list=[list(row) for row in amenity.collect()]
indexes = list(range(len(amenity_list)))

 

#split words and counts to different lists 
values = [r['count'] for r in amenity_list]
labels = [r['word'] for r in amenity_list]

 

#Plotting
bar_width = 0.1

 

pyplot.bar(indexes, values)

 

#add labels
labelidx = [i + bar_width for i in indexes] 
pyplot.xticks(labelidx, labels)
pyplot.show()


#### Firstly, challenge we faced was cleaning amenities column as in one listing we see Wifi and another listing we see "Wifi"(closed with "") but basically both are same. Took time for finding solution to it. Next is count of each amenity. Primarily we made use of UDF for this but we faced issues in saving dataframe and as well with assembler. We solved this by using for loop and this loop took lot of time and for safer side we saved this dataframe so that we could use this in future.

In [17]:
def getrows(df, rownums=None):
    return df.rdd.zipWithIndex().filter(lambda x: x[1] in rownums).map(lambda x: x[0])
  
def heat(col):
  check=col.split(',')
  if 'Heating' in check:
    return 1
  else: return 0
  
def ess(col):
  check=col.split(',')
  if 'Essentials' in check:
    return 1
  else: return 0
  
def kitchen(col):
  check=col.split(',')
  if 'Kitchen' in check:
    return 1
  else: return 0

 
def smoke(col):
  check=col.split(',')
  if 'Smoke detector' in check:
    return 1
  else: return 0
  
def wifi(col):
  check=col.split(',')
  if 'Wifi' in check:
    return 1
  else: return 0
    
 
 
for i in range(df.count()):
  list_heat.append((heat(getrows(df.select('amenities'),rownums=[i]).collect()[0][0])))
  list_wifi.append((wifi(getrows(df.select('amenities'),rownums=[i]).collect()[0][0])))
  list_ess.append((ess(getrows(df.select('amenities'),rownums=[i]).collect()[0][0])))
  list_kitchen.append((kitchen(getrows(df.select('amenities'),rownums=[i]).collect()[0][0])))
  list_smoke.append((smoke(getrows(df.select('amenities'),rownums=[i]).collect()[0][0])))
  
heat_df=spark.createDataFrame(list_heat, IntegerType())
wifi_df=spark.createDataFrame(list_wifi, IntegerType())
ess_df=spark.createDataFrame(list_ess, IntegerType())
kitchen_df=spark.createDataFrame(list_kitchen, IntegerType())
smoke_df=spark.createDataFrame(list_smoke, IntegerType())
 
df2=heat_df.rdd.zipWithIndex()
dfh=df2.map(lambda x: (list(x[0]) + [x[1]]))
df_heat=dfh.toDF(["Heat",'row_id'])

 

df2=wifi_df.rdd.zipWithIndex()
dfh=df2.map(lambda x: (list(x[0]) + [x[1]]))
df_wifi=dfh.toDF(["Wifi",'row_id'])

 

df2=ess_df.rdd.zipWithIndex()
dfh=df2.map(lambda x: (list(x[0]) + [x[1]]))
df_ess=dfh.toDF(["Essentials",'row_id'])

 

df2=kitchen_df.rdd.zipWithIndex()
dfh=df2.map(lambda x: (list(x[0]) + [x[1]]))
df_kitchen=dfh.toDF(["Kitchen",'row_id'])

 

df2=smoke_df.rdd.zipWithIndex()
dfh=df2.map(lambda x: (list(x[0]) + [x[1]]))
df_smoke=dfh.toDF(["Smoke Detector",'row_id'])

 


df=df.join(df_heat,on='rowid')
df=df.join(df_wifi,on='rowid')
df=df.join(df_ess,on='rowid')
df=df.join(df_kitchen,on='rowid')
df=df.join(df_smoke,on='rowid')
 


#### Created binary columns for all top 5 amenities. If a listing has Wifi as one of its amenity then Wifi column as 1 else 0 and similar concept for all other columns as well

In [19]:
import math

 

def distance(lat1,lon1,lat2,lon2):
    #lat1, lon1 = origin
    #lat2, lon2 = destination
    radius = 6371 # km

 

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

 

    return d
 
for i in range(df.count()):
  list_airport.append((distance(getrows(df.select('latitude'),rownums=[i]).collect()[0][0]),
  distance(getrows(df.select('longitude'),rownums=[i]).collect()[0][0]),
  52.3105,4.7683)) 
  
  list_citycenter.append((distance(getrows(df.select('latitude'),rownums=[i]).collect()[0][0]),
  distance(getrows(df.select('longitude'),rownums=[i]).collect()[0][0]),
  52.3792,4.8994)) 
  
airport_df=spark.createDataFrame(list_airport, FloatType())
df2=airport_df.rdd.zipWithIndex()
dfh=df2.map(lambda x: (list(x[0]) + [x[1]]))
df_airport=dfh.toDF(["Airport",'row_id'])

 

citycenter_df=spark.createDataFrame(list_citycenter, FloatType())
df2=airport_df.rdd.zipWithIndex()
dfh=df2.map(lambda x: (list(x[0]) + [x[1]]))
df_cc=dfh.toDF(["City Center",'row_id'])

 

dfnew=df.join(df_airport,on='rowid')
dfnew=df.join(df_cc,on='rowid')

#### Simialr problem of UDF was faced here as well. Hence, made use of for loop and join and saved dataframe so that we could use further

#Data Cleaning

In [22]:
df.count()

Out[5]: 20026

In [23]:
df.printSchema()

root
-- neighbourhood_cleansed: string (nullable = true)
-- city: string (nullable = true)
-- state: string (nullable = true)
-- zipcode: string (nullable = true)
-- host_id: integer (nullable = true)
-- host_since: string (nullable = true)
-- host_is_superhost: string (nullable = true)
-- host_listings_count: string (nullable = true)
-- host_total_listings_count: string (nullable = true)
-- host_verifications: string (nullable = true)
-- host_has_profile_pic: string (nullable = true)
-- host_identity_verified: string (nullable = true)
-- latitude: string (nullable = true)
-- longitude: string (nullable = true)
-- is_location_exact: string (nullable = true)
-- property_type: string (nullable = true)
-- room_type: string (nullable = true)
-- accommodates: integer (nullable = true)
-- bathrooms: string (nullable = true)
-- bedrooms: string (nullable = true)
-- beds: string (nullable = true)
-- bed_type: string (nullable = true)
-- amenities: string (nullable = true)
-- security_deposit: string (nullable = true)
-- cleaning_fee: string (nullable = true)
-- guests_included: integer (nullable = true)
-- extra_people: integer (nullable = true)
-- minimum_nights: integer (nullable = true)
-- maximum_nights: integer (nullable = true)
-- minimum_minimum_nights: integer (nullable = true)
-- maximum_minimum_nights: double (nullable = true)
-- minimum_maximum_nights: integer (nullable = true)
-- maximum_maximum_nights: string (nullable = true)
-- minimum_nights_avg_ntm: string (nullable = true)
-- maximum_nights_avg_ntm: double (nullable = true)
-- calendar_updated: string (nullable = true)
-- has_availability: string (nullable = true)
-- availability_30: integer (nullable = true)
-- availability_60: integer (nullable = true)
-- availability_90: string (nullable = true)
-- availability_365: string (nullable = true)
-- calendar_last_scraped: string (nullable = true)
-- instant_bookable: string (nullable = true)
-- is_business_travel_ready: string (nullable = true)
-- cancellation_policy: string (nullable = true)
-- require_guest_profile_picture: string (nullable = true)
-- require_guest_phone_verification: string (nullable = true)
-- calculated_host_listings_count: integer (nullable = true)
-- calculated_host_listings_count_entire_homes: integer (nullable = true)
-- calculated_host_listings_count_private_rooms: integer (nullable = true)
-- calculated_host_listings_count_shared_rooms: double (nullable = true)
-- number_of_reviews: double (nullable = true)
-- price: string (nullable = true)
-- longitude.1: double (nullable = true)
-- latitude.1: double (nullable = true)
-- neighbourhood: string (nullable = true)
-- n: integer (nullable = true)
-- score: string (nullable = true)
-- Wifi: string (nullable = true)
-- Heating: string (nullable = true)
-- Essentials: string (nullable = true)
-- Kitchen: string (nullable = true)
-- Smoke detector: string (nullable = true)
-- citycentre: double (nullable = true)
-- airport: double (nullable = true)
-- log_of_price: double (nullable = true)

In [24]:
df=df.select("accommodates","bathrooms","bedrooms","beds","price","neighbourhood_cleansed","property_type","bed_type","security_deposit","cleaning_fee","guests_included","extra_people","number_of_reviews","room_type","host_is_superhost","host_listings_count","score","Wifi","Heating","Essentials","Kitchen","Smoke detector",'citycentre','airport')

In [25]:

df=df.withColumn('Wifi',col('Wifi').cast('int'))
df=df.withColumn('Heating',col('Heating').cast('int'))
df=df.withColumn('Essentials',col('Essentials').cast('int'))
df=df.withColumn('Kitchen',col('Kitchen').cast('int'))
df=df.withColumn('Smoke detector',col('Smoke detector').cast('int'))
df=df.withColumn('bathrooms',col('bathrooms').cast('int'))
df=df.withColumn('bedrooms',col('bedrooms').cast('int'))
df=df.withColumn('beds',col('beds').cast('int'))
df=df.withColumn('security_deposit',col('security_deposit').cast('double'))
df=df.withColumn('cleaning_fee',col('cleaning_fee').cast('double'))
df=df.withColumn('score',col('score').cast('int'))
df=df.withColumn('price',col('price').cast('int'))

In [26]:
# Findings number of nulls in each column
# reference: https://stackoverflow.com/questions/44627386/how-to-find-count-of-null-and-nan-values-for-each-column-in-a-pyspark-dataframe
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+------------+---------+--------+----+-----+----------------------+-------------+--------+----------------+------------+---------------+------------+-----------------+---------+-----------------+-------------------+-----+----+-------+----------+-------+--------------+----------+-------+
accommodates|bathrooms|bedrooms|beds|price|neighbourhood_cleansed|property_type|bed_type|security_deposit|cleaning_fee|guests_included|extra_people|number_of_reviews|room_type|host_is_superhost|host_listings_count|score|Wifi|Heating|Essentials|Kitchen|Smoke detector|citycentre|airport|
+------------+---------+--------+----+-----+----------------------+-------------+--------+----------------+------------+---------------+------------+-----------------+---------+-----------------+-------------------+-----+----+-------+----------+-------+--------------+----------+-------+
 1| 8| 15| 33| 2| 0| 1| 1| 6018| 3604| 1| 1| 1| 1| 159| 1| 76| 17| 17| 17| 17| 18| 2| 2|
+------------+---------+--------+----+-----+----------------------+-------------+--------+----------------+------------+---------------+------------+-----------------+---------+-----------------+-------------------+-----+----+-------+----------+-------+--------------+----------+-------+

#There are around 10000 nulls in security deposit and cleaning fee combined which is around half of our data set. So let us not consider there columns in our model.

In [28]:
df=df.drop('security_deposit','cleaning_fee')
df.count()

Out[10]: 20026

In [29]:
#Ref - https://blog.zhaytam.com/2019/07/15/outliers-detection-in-pyspark-2-interquartile-range/
def calculate_bounds(dataframe):
  bounds = {c: dict(zip(["q1", "q3"], dataframe.approxQuantile(c, [0.25, 0.75], 0)))
    for c,d in zip(dataframe.columns, dataframe.dtypes) if d[1] == "int"
    }
  for c in bounds:
    iqr = bounds[c]['q3'] - bounds[c]['q1']
    bounds[c]['min'] = bounds[c]['q1'] - (iqr * 1.5)
    bounds[c]['max'] = bounds[c]['q3'] + (iqr * 1.5)
  return bounds
    

In [30]:
df_price=df.select('price')
calculate_bounds(df_price)

Out[12]: {'price': {'q1': 99.0, 'q3': 185.0, 'min': -30.0, 'max': 314.0}}

In [31]:
splits = [0.0,314, 1000.0, 3000.0, 5000.0, 7000.0,float("inf")]
bucketizer = Bucketizer(splits=splits, inputCol="price", outputCol="bucketedFeatures")
bucketedData = bucketizer.transform(df_price)
bucketedData.groupBy("bucketedFeatures").count().show()

+----------------+-----+
bucketedFeatures|count|
+----------------+-----+
 0.0|18875|
 null| 2|
 1.0| 1082|
 4.0| 2|
 3.0| 3|
 2.0| 56|
 5.0| 6|
+----------------+-----+

In [32]:
df=df.filter('price <=314')
df.count()

Out[14]: 18877

In [33]:
df=df.dropna(how='any')
df.count()

Out[15]: 18614

In [34]:
df_DS=df.select("accommodates","bathrooms","bedrooms","beds","price","guests_included","extra_people","number_of_reviews","host_listings_count")

df_DS.describe().toPandas().transpose()


,0,1,2,3,4
summary,count,mean,stddev,min,max
accommodates,18614,2.76431718061674,1.1237572030961747,1,16
bathrooms,18614,1.051198022993446,0.2729015838910834,0,8
bedrooms,18614,1.3887933813258837,0.817766715168559,0,12
beds,18614,1.6883528526915226,1.1827376119246986,0,32
price,18614,139.54561083055765,58.84286278054279,9,314
guests_included,18614,1.4228537659825937,0.8140600156778348,1,16
extra_people,18614,11.221929730310519,23.20886665346957,0,280
number_of_reviews,18614,25.236327495433546,52.99942847272882,0.0,821.0
host_listings_count,18614,1.9534758783711186,8.688326054130359,0,9


In [35]:
import six
df_Corr=df.select("accommodates","bathrooms","bedrooms","beds","price","guests_included","extra_people","number_of_reviews","host_listings_count","airport","citycentre")

for i in df_Corr.columns:
    if not( isinstance(df_Corr.select(i).take(1)[0][0], six.string_types)):
        print( "Correlation to PRICE for ", i, df_Corr.stat.corr('price',i))

Correlation to PRICE for accommodates 0.4742533993794424
Correlation to PRICE for bathrooms 0.2015603104306123
Correlation to PRICE for bedrooms 0.4170371480830313
Correlation to PRICE for beds 0.3561492461123143
Correlation to PRICE for price 1.0
Correlation to PRICE for guests_included 0.20968145885776965
Correlation to PRICE for extra_people 0.07715305869478054
Correlation to PRICE for number_of_reviews -0.12192000325643704
Correlation to PRICE for airport -0.004963346487779615
Correlation to PRICE for citycentre -0.18211646214218413

In [36]:
# reference: https://stackoverflow.com/questions/55546467/how-to-plot-correlation-heatmap-when-using-pysparkdatabricks

neighbourhood_indexer = StringIndexer(inputCol='neighbourhood_cleansed',outputCol='neighbourhood_index',handleInvalid='keep')
propertyType_Indexer = StringIndexer(inputCol='property_type',outputCol='property_type_index',handleInvalid='keep')
bedType_Indexer = StringIndexer(inputCol='bed_type',outputCol='bed_type_index',handleInvalid='keep')
roomType_indexer = StringIndexer(inputCol='room_type',outputCol='room_type_index',handleInvalid='keep')
hostissuperhost_indexer = StringIndexer(inputCol='host_is_superhost',outputCol='hostisusperhost_index',handleInvalid='keep')


# Vector assembler is used to create a vector of input features

assembler_allVariables=VectorAssembler(inputCols=['accommodates','bathrooms','bedrooms','beds','number_of_reviews','neighbourhood_index','property_type_index','bed_type_index',
                                       'room_type_index','hostisusperhost_index',"Wifi","Heating","Essentials","Kitchen","Smoke detector",'airport','citycentre','price'],
                            outputCol="features")

# Pipeline is used to pass the data through indexer and assembler simultaneously. Also, it helps to pre-rocess the test data
# in the same way as that of the train data

pipe = Pipeline(stages=[neighbourhood_indexer,propertyType_Indexer,bedType_Indexer,roomType_indexer,hostissuperhost_indexer,assembler_allVariables])

 

fit_pipe = pipe.fit(df)
df_correlated=fit_pipe.transform(df)

 

# Getting the correlation dense matrix
corr = Correlation.corr(df_correlated, "features").collect()[0][0]
# Coverting it to a list so that it can plotted
corrmatrix = corr.toArray().tolist() 
# defining the features data columns since if convert the corr matrix to list we will loose column headers
data_columns=[
 'accommodates','bathrooms','bedrooms','beds','number_of_reviews','neighbourhood_index','property_type_index','bed_type_index',
                                       'room_type_index','hostisusperhost_index',"Wifi","Heating","Essentials","Kitchen","Smoke detector",'airport','citycentre','price']
# We create a dataframe with correlated values list between features and features headers
corrdf=spark.createDataFrame(corrmatrix,data_columns)

# We will plot the correlation matrix with matplot matrix show.

fig, ax = plt.subplots()
# Creating a matrix plot with correlation matrix with max value 1 and min value -1
corr_ax=ax.matshow(corrmatrix,vmax=1,vmin=-1)
# Setting title xlabel , ylabel, xticks and yticks
ax.set_title("Correlation Matrix")
ax.set_xticks(np.arange(len(data_columns)))
ax.set_yticks(np.arange(len(data_columns)))
ax.set_xticklabels(data_columns)
ax.set_yticklabels(data_columns)
plt.gca().xaxis.tick_bottom()
plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
plt.tick_params(axis="both", which="both" )
fig.colorbar(corr_ax)

#fig.tight_layout()
display(fig)




In [37]:
from pyspark.sql.functions import lit
from math import log
df=df.withColumn("log_of_price",log10(df.price))



In [38]:
from pyspark.sql.functions import lit
from math import log
df=df.withColumn("log_of_price",log10(df.price))


df1=df.select('price','log_of_price').toPandas()
%matplotlib inline
import matplotlib.pyplot as plt
df1.hist(column='price')
df1.hist(column='log_of_price')


In [39]:

train_data_LR,test_data_LR=df.randomSplit([0.7,0.3],seed=12345)
  
  

#Indexing the Categorical variable
neighbourhood_indexer = StringIndexer(inputCol='neighbourhood_cleansed',outputCol='neighbourhood_index',handleInvalid='keep')
propertyType_Indexer = StringIndexer(inputCol='property_type',outputCol='property_type_index',handleInvalid='keep')
bedType_Indexer = StringIndexer(inputCol='bed_type',outputCol='bed_type_index',handleInvalid='keep')
roomType_indexer = StringIndexer(inputCol='room_type',outputCol='room_type_index',handleInvalid='keep')
hostissuperhost_indexer = StringIndexer(inputCol='host_is_superhost',outputCol='hostisusperhost_index',handleInvalid='keep')


# Vector assembler is used to create a vector of input features

assembler_allVariables=VectorAssembler(inputCols=['accommodates','bathrooms','bedrooms','beds','number_of_reviews','neighbourhood_index','property_type_index','bed_type_index',
                                       'room_type_index','hostisusperhost_index',"Wifi","Heating","Essentials","Kitchen","Smoke detector",'airport','citycentre'],
                            outputCol="features")

# Pipeline is used to pass the data through indexer and assembler simultaneously. Also, it helps to pre-rocess the test data
# in the same way as that of the train data

pipe = Pipeline(stages=[neighbourhood_indexer,propertyType_Indexer,bedType_Indexer,roomType_indexer,hostissuperhost_indexer,assembler_allVariables])

fitted_pipe=pipe.fit(train_data_LR)

train_data_LR=fitted_pipe.transform(train_data_LR)

# Create an object for the Linear Regression model

lr_model = LinearRegression(labelCol='log_of_price')

# Fit the model on the train data

fit_model = lr_model.fit(train_data_LR.select(['features','log_of_price']))

# Transform the test data using the model to predict the duration

test_data_LR=fitted_pipe.transform(test_data_LR)

# Store the results in a dataframe

results = fit_model.transform(test_data_LR)

results.select(['log_of_price','prediction']).show()

+------------------+------------------+
 log_of_price| prediction|
+------------------+------------------+
1.8976270912904414|1.9210484432449901|
1.9493900066449128|1.9205230375195164|
1.9493900066449128|1.9204526496123393|
1.9777236052888478|1.9390392379123256|
1.6532125137753437| 1.877174445818467|
1.7481880270062005|1.8699700965304986|
1.8325089127062364|1.8648912578884582|
1.2787536009528289|1.8715132431311228|
1.4313637641589874|1.8945740324777682|
1.6020599913279623|1.9957862026401592|
1.6020599913279623|1.8469383946939606|
1.6127838567197355|1.9127596453579832|
1.6901960800285136|1.9103565250802783|
1.6901960800285136| 2.067388152509213|
1.6989700043360187|1.7935547739140631|
1.7481880270062005| 1.923369114425604|
1.7634279935629373|1.8780759654196164|
1.7708520116421442|1.8625370957639116|
1.7781512503836436|1.9367711837994437|
1.7781512503836436|1.8024981888347504|
+------------------+------------------+
only showing top 20 rows

In [40]:
test_results = fit_model.evaluate(test_data_LR)
test_results.residuals.show()

+--------------------+
 residuals|
+--------------------+
-0.02342135195454...|
0.028866969125396436|
0.028937357032573496|
0.038684367376522255|
 -0.2239619320431232|
-0.12178206952429815|
-0.03238234518222183|
 -0.5927596421782939|
-0.46321026831878087|
-0.39372621131219687|
 -0.2448784033659983|
 -0.2999757886382477|
-0.22016044505176469|
 -0.3771920724806994|
 -0.0945847695780444|
-0.17518108741940353|
-0.11464797185667908|
-0.09168508412176735|
-0.15861993341580005|
-0.02434693845110...|
+--------------------+
only showing top 20 rows

In [41]:
test_results.rootMeanSquaredError

Out[34]: 0.14770996411233014

### Here RMSE is for Log of Price, so for price the Root mean squared error will be exponential of the above value which is 15%. It means that typically our original values deviate by 15% from geometric mean.

In [43]:
test_results.r2

Out[35]: 0.3896591409259127

In [44]:
# Print the coefficients and intercept for linear regression
print("Coefficients: %s" % str(fit_model.coefficients))
print("Intercept: %s" % str(fit_model.intercept))

# Summarize the model over the training set and print out some metrics
trainingSummary = fit_model.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

Coefficients: [0.05673029898320129,0.028099663518386848,0.03533065643209354,-0.007444152944353079,-0.000262379037705157,-0.004888549395834618,0.008636643213557098,-0.030648337197597384,-0.10791797615375039,0.03329683036116999,0.02260305367228644,0.02260305367228644,0.047917995498599875,-0.002024317933555641,0.023958912255567007,-0.002718325042445774,-0.01983520445504557]
Intercept: 1.9097925736311203
numIterations: 21
objectiveHistory: [0.4999999999999929, 0.3331404003593619, 0.3161943418009159, 0.3086355195713608, 0.3057588019257409, 0.3046658428919855, 0.3043019834142413, 0.30420257670641604, 0.3041612251278636, 0.30415144636865676, 0.30415081105892483, 0.304150696846726, 0.30415062393655745, 0.30415058593098365, 0.3041505847347423, 0.30415058366266123, 0.30415058363098524, 0.30415058362487457, 0.30415058362094527, 0.3041505836204834, 0.3041505836198155]
+--------------------+
 residuals|
+--------------------+
 -0.3434825744423686|
 -0.2946286027945142|
-0.13890763318604527|
-0.09707258294565535|
0.054395984086382354|
 0.04013796445003592|
 -0.4157163571326159|
-0.22710826804758577|
 -0.1446143946823053|
0.009199297977119159|
 -0.350972142871645|
 -0.0583538276559048|
 -0.0871512461753976|
 -0.5934108645744007|
 -0.0473733081224621|
-0.43146864050535694|
 -0.1903311225144375|
-0.27525114959052654|
-0.43540563422471257|
-0.29950524424550684|
+--------------------+
only showing top 20 rows

RMSE: 0.147478
r2: 0.391699

#### From the coefficients we can interpret that if a customer needs WIFI as an amenity then the price of the listing will be 2.5 % more than a listings that does not have WIFI as an amenity.

The same with other amenities as well
2% for Heating
4.7% for essentials

In [46]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
pd_df_lr = results.toPandas()
fig, ax=plt.subplots(figsize=(10,6))
ax=sns.regplot(pd_df_lr['prediction'],pd_df_lr['log_of_price'])
display(plt.show())

In [47]:
train_data_RF,test_data_RF=df.randomSplit([0.7,0.3],seed=12345)



# Creating Random Forest Regressor model object
rf_model=RandomForestRegressor(featuresCol="features",labelCol='log_of_price',maxBins=500)


#Creating the pipeline object with Pipeline stages for transforming the data and applying model
pipe = Pipeline(stages=[neighbourhood_indexer,propertyType_Indexer,bedType_Indexer,roomType_indexer,hostissuperhost_indexer,assembler_allVariables])

fitted_pipe=pipe.fit(train_data_RF)

train_data_RF=fitted_pipe.transform(train_data_RF)

# Fit the model on the train data

fit_model = rf_model.fit(train_data_RF.select(['features','log_of_price']))

# Transform the test data using the model to predict the duration

test_data_RF=fitted_pipe.transform(test_data_RF)

# Store the results in a dataframe

results = fit_model.transform(test_data_RF)

results.select(['log_of_price','prediction']).show()



+------------------+------------------+
 log_of_price| prediction|
+------------------+------------------+
1.8976270912904414|1.8997303726861818|
1.9493900066449128|1.8997303726861818|
1.9493900066449128|1.8997303726861818|
1.9777236052888478|1.9936371737923273|
1.6532125137753437|1.9133020891312225|
1.7481880270062005|1.7753069719330914|
1.8325089127062364| 1.777363828624824|
1.2787536009528289|1.8088926969604107|
1.4313637641589874|1.8669670406890457|
1.6020599913279623|1.8827716744573575|
1.6020599913279623|1.8410482082945248|
1.6127838567197355|1.8777974578220367|
1.6901960800285136|1.8949317296245862|
1.6901960800285136| 2.047452727235213|
1.6989700043360187| 1.828613254446795|
1.7481880270062005|1.8944097339347512|
1.7634279935629373|1.8146679208549927|
1.7708520116421442|1.8458574472897642|
1.7781512503836436|1.8588292974484468|
1.7781512503836436|1.8349013668293874|
+------------------+------------------+
only showing top 20 rows

In [48]:
# Evaluating Decision Tree regression model based on the Regression Evaluator.
evaluate = RegressionEvaluator(
    labelCol="log_of_price", predictionCol="prediction", metricName="rmse")

# Calculating the root mean square value for the model.
rmse = evaluate.evaluate(results)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 0.141496

In [49]:
# Creating a another Regression Evaluator object for calculating r2 score.
evaluate1 = RegressionEvaluator(
    labelCol="log_of_price", predictionCol="prediction", metricName="r2")

r2 = evaluate1.evaluate(results)
print("R Squared (R2) on test data = %g" % r2)

R Squared (R2) on test data = 0.43993

In [50]:



import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
pd_df_lr = results.toPandas()
fig, ax=plt.subplots(figsize=(10,6))
ax=sns.regplot(pd_df_rf['prediction'],pd_df_rf['log_of_price'])
display(plt.show())